In [2]:
import time
start_time = time.time()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from math import sin, cos, sqrt, atan2, radians, acos

In [3]:
data = pd.read_csv(r'C:\Users\INFO-DSK-04\Desktop\Asset_Data\SQL Server\Data_354972062885746.csv')
# data.head()

In [4]:
# data.info()

In [5]:
data.rename(columns={"TrackDateTime_new":"TrackDateTime"},inplace=True)

In [6]:
# sns.distplot(data['Speed'], hist=True, kde=True, 
#              bins=int(180/5), color = 'darkblue', 
#              hist_kws={'edgecolor':'black'},
#              kde_kws={'linewidth': 4})

In [7]:
#Function for changing the syntax of datetime to a paticular format
new_format = "%Y:%m:%d %H:%M:%S"
def change_datetime_syntax(date_text):
    try:
        d=datetime.strptime( date_text , '%d-%m-%Y %H:%M:%S.%f') 
        date_text=d.strftime(new_format)
        return date_text
    except:  
        try:
            d=datetime.strptime(date_text , '%d-%m-%Y %H:%M:%S') 
            date_text=d.strftime(new_format)
            return date_text
        except:
            try:
                d=datetime.strptime(date_text , '%Y:%m:%d %H:%M:%S.%f') 
                date_text=d.strftime(new_format)
                return date_text
            except:
                try:
                    d=datetime.strptime(date_text , '%Y:%m:%d %H:%M:%S') 
                    date_text=d.strftime(new_format)
                    return date_text
                except:
                    try:
                        d=datetime.strptime(date_text , '%Y-%m-%d %H:%M:%S.%f') 
                        date_text=d.strftime(new_format)
                        return date_text
                    except:
                        try:
                            d=datetime.strptime(date_text , '%Y-%m-%d %H:%M:%S') 
                            date_text=d.strftime(new_format)
                            return date_text
                        except:
                            print(date_text,"New format found, include this in function")


In [8]:
data['TrackDateTime_new']=[change_datetime_syntax(k) for k in data.TrackDateTime]
data.drop("TrackDateTime",axis=1,inplace=True)
data.rename(columns={"TrackDateTime_new":"TrackDateTime"},inplace=True)


In [9]:
data = data.sort_values('TrackDateTime')
data.reset_index(inplace=True)
data = data.drop('index', axis=1)

In [10]:
data.columns

Index(['SlNo', 'AssetNo', 'DeviceNo', 'Latitude', 'Longitude', 'Altitude',
       'Speed', 'Satellite', 'Direction', 'Location', 'GeofenceName',
       'MessageNumber', 'OdoMeter', 'Ignition', 'Digital1', 'Digital2',
       'Digital3', 'Digital4', 'Analog1', 'Analog2', 'SoftwareVersion',
       'RoadSpeed', 'DriverId', 'FuelinLtrs', 'EngineRpm', 'IbuttonId',
       'UpdationTime', 'SqncNumber', 'OdometerLL', 'AdditionalData',
       'TrackDateTime'],
      dtype='object')

In [11]:
# To check if the datetime is in ascending order
FMT = '%Y:%m:%d %H:%M:%S'

for i in range(1, len(data.TrackDateTime), 1):
    x = datetime.strptime(str(data.TrackDateTime.iloc[i]), FMT) - datetime.strptime(str(data.TrackDateTime.iloc[i-1]), FMT)
    if int(x.total_seconds()) < 0:
        # If it doesn't print anything then datetime is in ascending order
        print(i,int(x.total_seconds()))

In [12]:
R = 6371 # Radius of the earth in km
dist = [0]
for i in range(1, len(data.Longitude), 1):
    dLat = radians(data.Latitude.iloc[i] - data.Latitude.iloc[i-1])
    dLon = radians(data.Longitude.iloc[i] - data.Longitude.iloc[i-1])
    rLat1 = radians(data.Latitude.iloc[i-1])
    rLat2 = radians(data.Latitude.iloc[i])
    a = sin(dLat/2) * sin(dLat/2) + cos(rLat1) * cos(rLat2) * sin(dLon/2) * sin(dLon/2) 
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c
    dist.append(d)

In [13]:
data['distance'] = dist

In [14]:
data['total_distance'] = pd.Series(dist).cumsum().round(2)

In [15]:
def angle_between_vectors_degrees(u, v):
    """Return the angle between two vectors in any dimension space,
    in degrees."""
    return np.degrees(
        acos(np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))))

In [16]:
angle1 = [np.nan]

for i in range(1,len(data['Latitude']),1):   
    if i != len(data['Latitude'])-1:
        # The points in tuple Latitude/Longitudeg degrees space
        A = [data.Latitude.iloc[i-1],data.Longitude.iloc[i-1]]
        B = [data.Latitude.iloc[i],data.Longitude.iloc[i]]
        C = [data.Latitude.iloc[i+1],data.Longitude.iloc[i+1]]
        try:
            if A == B == C:
                angle1.append(0)
                
            elif ((A==B) | (B==C)):
                angle1.append(0)
                
            else:
                # Convert the points to numpy Latitudeitude/Longitudegitude radians space
                a = np.radians(np.array(A))
                b = np.radians(np.array(B))
                c = np.radians(np.array(C))
                
                # print(a,'\n',b,'\n',c,'\n\n')

                # Vectors in Latitudeitude/Longitudegitude space
                avec = a - b
                cvec = c - b

                # Adjust vectors for changed Longitudegitude scale at given Latitudeitude into 2D space
                lat_ = b[0]
                avec[1] *= cos(lat_)
                cvec[1] *= cos(lat_)

                # Find the angle between the vectors in 2D space
                angle2deg = angle_between_vectors_degrees(avec, cvec)
                angle1.append(angle2deg)

        except:
            angle1.append(0)
    else:
        angle1.append(np.nan)

In [17]:
data['Angle'] = angle1

## Sudden Acceleration

In [18]:
sudden_acc= [np.nan]

# print('{:7s}'.format('Index'),
#       '{:20s}'.format('Datetime i-1'), 
#       '{:15s}'.format('Speed for i-1'),
#       '{:20s}'.format('Datetime'), 
#       '{:13s}'.format('Speed for i'), 
#       '{:12s}'.format('Difference'),
#       '{:12s}'.format('Distance covered (m)'))


FMT = '%Y:%m:%d %H:%M:%S'

for i in range(1, len(data.Speed), 1):    
    x = datetime.strptime(str(data.TrackDateTime.iloc[i]), FMT) - datetime.strptime(str(data.TrackDateTime.iloc[i-1]), FMT)
    y = data.Speed.iloc[i] - data.Speed.iloc[i-1]
        
    if (((int(x.total_seconds()) > 0) & (int(x.total_seconds()) < 7)) & (data['distance'].iloc[i] < 0.025) & (y > 30)):
#         print('{:7s}'.format(str(i)),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])), 
#               '{:15s}'.format(str(data.Speed.iloc[i-1])),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i])), 
#               '{:13s}'.format(str(data.Speed.iloc[i])),
#               '{:12s}'.format(str(y.round(3))),
#               '{:12s}'.format(str(data.distance[i]*1000))
#              )
        sudden_acc.append(15)
        
    elif (((int(x.total_seconds()) > 0) & (int(x.total_seconds()) < 62)) & (data['distance'].iloc[i] < 0.03) & (y > 40)):
#         print('{:7s}'.format(str(i)),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])), 
#               '{:15s}'.format(str(data.Speed.iloc[i-1])),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i])), 
#               '{:13s}'.format(str(data.Speed.iloc[i])),
#               '{:12s}'.format(str(y.round(3))),
#               '{:12s}'.format(str(data.distance[i]*1000))
#              )
        sudden_acc.append(15)
            
       
    elif (((int(x.total_seconds()) > 0) & (int(x.total_seconds()) < 12)) & (data['distance'].iloc[i] < 0.04) & (y > 50)):
#         print('{:7s}'.format(str(i)),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])), 
#               '{:15s}'.format(str(data.Speed.iloc[i-1])),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i])), 
#               '{:13s}'.format(str(data.Speed.iloc[i])),
#               '{:12s}'.format(str(y.round(3))),
#               '{:12s}'.format(str(data.distance[i]*1000))
#              )
        sudden_acc.append(15)
            
    elif (((int(x.total_seconds()) > 0) & (int(x.total_seconds()) < 7) ) & (y > 30)):
#         print('{:7s}'.format(str(i)),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])), 
#               '{:15s}'.format(str(data.Speed.iloc[i-1])),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i])), 
#               '{:13s}'.format(str(data.Speed.iloc[i])),
#               '{:12s}'.format(str(y.round(3))),
#               '{:12s}'.format(str(data.distance[i]*1000))
#              )
        sudden_acc.append(15)
        
    else:
        sudden_acc.append(0)

## Harsh Braking

In [19]:
harsh_braking = [np.nan]

# print('{:7s}'.format('Index'),
#       '{:20s}'.format('Datetime i-1'), 
#       '{:15s}'.format('Speed for i-1'),
#       '{:20s}'.format('Datetime'), 
#       '{:13s}'.format('Speed for i'), 
#       '{:12s}'.format('Difference'),
#       '{:12s}'.format('Distance covered (m)'))


FMT = '%Y:%m:%d %H:%M:%S'

for i in range(1, len(data.Speed), 1):    
    x = datetime.strptime(str(data.TrackDateTime.iloc[i]), FMT) - datetime.strptime(str(data.TrackDateTime.iloc[i-1]), FMT)
    y = data.Speed.iloc[i] - data.Speed.iloc[i-1]

    if (((int(x.total_seconds()) > 0) & (int(x.total_seconds()) < 7)) & (data['distance'].iloc[i] < 0.025) & (y < -25)):
#         print('{:7s}'.format(str(i)),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])), 
#               '{:15s}'.format(str(data.Speed.iloc[i-1])),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i])), 
#               '{:13s}'.format(str(data.Speed.iloc[i])),
#               '{:12s}'.format(str(y.round(3))),
#               '{:12s}'.format(str(data.distance[i]*1000))
#              )
        
        harsh_braking.append(14)
        
    elif (((int(x.total_seconds()) > 0) & (int(x.total_seconds()) < 7)) & (y < -30)):
#         print('{:7s}'.format(str(i)),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])), 
#               '{:15s}'.format(str(data.Speed.iloc[i-1])),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i])), 
#               '{:13s}'.format(str(data.Speed.iloc[i])),
#               '{:12s}'.format(str(y.round(3))),
#               '{:12s}'.format(str(data.distance[i]*1000))
#              )
        
        harsh_braking.append(14)
        
    elif (((int(x.total_seconds()) > 0) & (int(x.total_seconds()) < 22)) & (data['distance'].iloc[i] < 0.03) & (y < -40)):
#         print('{:7s}'.format(str(i)),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])), 
#               '{:15s}'.format(str(data.Speed.iloc[i-1])),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i])), 
#               '{:13s}'.format(str(data.Speed.iloc[i])),
#               '{:12s}'.format(str(y.round(3))),
#               '{:12s}'.format(str(data.distance[i]*1000))
#              )
        
        harsh_braking.append(14)
        
    elif (((int(x.total_seconds()) > 0) & (int(x.total_seconds()) < 62)) & (data['distance'].iloc[i] < 0.025) & (y < -40)):
#         print('{:7s}'.format(str(i)),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])), 
#               '{:15s}'.format(str(data.Speed.iloc[i-1])),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i])), 
#               '{:13s}'.format(str(data.Speed.iloc[i])),
#               '{:12s}'.format(str(y.round(3))),
#               '{:12s}'.format(str(data.distance[i]*1000))
#              )
        
        harsh_braking.append(14)
       
    elif (((int(x.total_seconds()) > 0) & (int(x.total_seconds()) < 62)) & (data['distance'].iloc[i] < 0.04) & (y < -35) & (data.Speed.iloc[i] > 50)):
#         print('{:7s}'.format(str(i)),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])), 
#               '{:15s}'.format(str(data.Speed.iloc[i-1])),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i])), 
#               '{:13s}'.format(str(data.Speed.iloc[i])),
#               '{:12s}'.format(str(y.round(3))),
#               '{:12s}'.format(str(data.distance[i]*1000))
#                  )
        harsh_braking.append(14)
                   
    elif  ((data['distance'].iloc[i] < 0.01) & (y < -25)):
#         print('{:7s}'.format(str(i)),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])), 
#               '{:15s}'.format(str(data.Speed.iloc[i-1])),
#               '{:20s}'.format(str(data.TrackDateTime.iloc[i])), 
#               '{:13s}'.format(str(data.Speed.iloc[i])),
#               '{:12s}'.format(str(y.round(3))),
#               '{:12s}'.format(str(data.distance[i]*1000))
#              )
        
        harsh_braking.append(14)
            
    else:
        harsh_braking.append(0)

## Rash Turning

In [20]:
rash_turn = [np.nan]

# print('{:7s}'.format('Index'),
#       '{:20s}'.format('Datetime i-1'),
#       '{:20s}'.format('Datetime'), 
#       '{:13s}'.format('Speed for i'), 
#       '{:12s}'.format('Angle'),
#       '{:15s}'.format('Alert'),
#       '{:12s}'.format('Distance covered'))


FMT = '%Y:%m:%d %H:%M:%S'

for i in range(1, len(data.Speed), 1):
    
    x = datetime.strptime(str(data.TrackDateTime.iloc[i]), FMT) - datetime.strptime(str(data.TrackDateTime.iloc[i-1]), FMT)    
    if ((int(x.total_seconds()) > 0) & (int(x.total_seconds()) < 6) & (data['distance'].iloc[i] < 0.02)):        
        
        if ((data['Angle'].iloc[i] > 0) & (data['Angle'].iloc[i] <= 10) & (data['Speed'].iloc[i] > 7)):
#             print('{:07d}'.format(i),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i])),
#                   '{:13s}'.format(str(data.Speed.iloc[i])),
#                   '{:12s}'.format(str(round(data.Angle.iloc[i],3))),
#                   '{:15s}'.format('Rash Turning'),
#                   '{:12s}'.format(str(round(data.distance[i],3)))
#                  )
            rash_turn.append(32)
            
        elif ((data['Angle'].iloc[i] > 10) & (data['Angle'].iloc[i] <= 50) & (data['Speed'].iloc[i] > 10)):            
#             print('{:07d}'.format(i),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i])),
#                   '{:13s}'.format(str(data.Speed.iloc[i])),
#                   '{:12s}'.format(str(round(data.Angle.iloc[i],3))),
#                   '{:15s}'.format('Rash Turning'),
#                   '{:12s}'.format(str(round(data.distance[i],3)))
#                  )
            rash_turn.append(32)
            
        elif ((data['Angle'].iloc[i] > 50) & (data['Angle'].iloc[i] <= 90) & (data['Speed'].iloc[i] > 13)):            
#             print('{:07d}'.format(i),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i])),
#                   '{:13s}'.format(str(data.Speed.iloc[i])),
#                   '{:12s}'.format(str(round(data.Angle.iloc[i],3))),
#                   '{:15s}'.format('Rash Turning'),
#                   '{:12s}'.format(str(round(data.distance[i],3)))
#                  )
            rash_turn.append(32)    
        
        
        elif ((data['Angle'].iloc[i] > 90) & (data['Angle'].iloc[i] <= 120) & (data['Speed'].iloc[i] > 15)):            
#             print('{:07d}'.format(i),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i])),
#                   '{:13s}'.format(str(data.Speed.iloc[i])),
#                   '{:12s}'.format(str(round(data.Angle.iloc[i],3))),
#                   '{:15s}'.format('Rash Turning'),
#                   '{:12s}'.format(str(round(data.distance[i],3)))
#                  )
            rash_turn.append(32)
        
        elif ((data['Angle'].iloc[i] > 120) & (data['Angle'].iloc[i] <= 150) & (data['Speed'].iloc[i] > 22)):            
#             print('{:07d}'.format(i),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i])),
#                   '{:13s}'.format(str(data.Speed.iloc[i])),
#                   '{:12s}'.format(str(round(data.Angle.iloc[i],3))),
#                   '{:15s}'.format('Rash Turning'),
#                   '{:12s}'.format(str(round(data.distance[i],3)))
#                  )
            rash_turn.append(32)
        
        elif ((data['Angle'].iloc[i] > 150) & (data['Angle'].iloc[i] <= 160) & (data['Speed'].iloc[i] > 28)):            
#             print('{:07d}'.format(i),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i])),
#                   '{:13s}'.format(str(data.Speed.iloc[i])),
#                   '{:12s}'.format(str(round(data.Angle.iloc[i],3))),
#                   '{:15s}'.format('Rash Turning'),
#                   '{:12s}'.format(str(round(data.distance[i],3)))
#                  )
            rash_turn.append(32)
        
        
        elif ((data['Angle'].iloc[i] > 160) & (data['Angle'].iloc[i] <= 170) & (data['Speed'].iloc[i] > 35)):            
#             print('{:07d}'.format(i),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i-1])),
#                   '{:20s}'.format(str(data.TrackDateTime.iloc[i])),
#                   '{:13s}'.format(str(data.Speed.iloc[i])),
#                   '{:12s}'.format(str(round(data.Angle.iloc[i],3))),
#                   '{:15s}'.format('Rash Turning'),
#                   '{:12s}'.format(str(round(data.distance[i],3)))
#                  )
            rash_turn.append(32)
        
        else:
            rash_turn.append(0)
        
    else:
        rash_turn.append(0)

In [21]:
data['Sudden_Acceleration'] = sudden_acc
data['Harsh_Braking'] = harsh_braking
data['Rash_Turning'] = rash_turn

## Driver Scorecard

In [22]:
ds = []

for i in range(0, len(data)):
#     x = datetime.strptime(str(data.TrackDateTime.iloc[i]), FMT) - datetime.strptime(str(data.TrackDateTime.iloc[i-1]), FMT)
    if i == 0:
        ds.append(100)
        
    elif ((data['Sudden_Acceleration'].iloc[i] == 14) & (data['Rash_Turning'].iloc[i] == 32)):
        ds.append(ds[i-1] - 10)
        
    elif ((data['Harsh_Braking'].iloc[i] == 15) & (data['Rash_Turning'].iloc[i] == 32)):
        ds.append(ds[i-1] - 8)
        
    elif (data['Rash_Turning'].iloc[i] == 32):
        ds.append(ds[i-1] - 5)
        
    elif ((data['Sudden_Acceleration'].iloc[i] == 14) | (data['Harsh_Braking'].iloc[i] == 15)):
        ds.append(ds[i-1] - 3)
        
    else:
        ds.append(ds[i-1])

In [23]:
data['Driver_scorecard'] = ds

In [24]:
Driver_Scorecard = data[['AssetNo',
                         'DeviceNo',
                         'TrackDateTime',
                         'Longitude',
                         'Latitude',
                         'Sudden_Acceleration',
                         'Harsh_Braking',
                         'Rash_Turning',
                         'Driver_scorecard']]

In [25]:
Alerts = Driver_Scorecard[(Driver_Scorecard['Sudden_Acceleration'] == 14) |
                          (Driver_Scorecard['Harsh_Braking'] == 15) |
                          (Driver_Scorecard['Rash_Turning'] == 32)]

In [26]:
data.columns

Index(['SlNo', 'AssetNo', 'DeviceNo', 'Latitude', 'Longitude', 'Altitude',
       'Speed', 'Satellite', 'Direction', 'Location', 'GeofenceName',
       'MessageNumber', 'OdoMeter', 'Ignition', 'Digital1', 'Digital2',
       'Digital3', 'Digital4', 'Analog1', 'Analog2', 'SoftwareVersion',
       'RoadSpeed', 'DriverId', 'FuelinLtrs', 'EngineRpm', 'IbuttonId',
       'UpdationTime', 'SqncNumber', 'OdometerLL', 'AdditionalData',
       'TrackDateTime', 'distance', 'total_distance', 'Angle',
       'Sudden_Acceleration', 'Harsh_Braking', 'Rash_Turning',
       'Driver_scorecard'],
      dtype='object')

In [27]:
data.loc[675:680][['DeviceNo','Latitude', 'Longitude','distance','TrackDateTime','Speed','Sudden_Acceleration', 'Harsh_Braking', 'Rash_Turning']]

,DeviceNo,Latitude,Longitude,distance,TrackDateTime,Speed,Sudden_Acceleration,Harsh_Braking,Rash_Turning
675,354972062885746,19.12104,73.01676,0.028115,2019:09:25 11:23:28,72,0.0,0.0,0.0
676,354972062885746,19.12083,73.01679,0.023563,2019:09:25 11:23:33,50,0.0,0.0,0.0
677,354972062885746,19.12078,73.01680,0.005658,2019:09:25 11:23:38,4,0.0,14.0,0.0
678,354972062885746,19.12066,73.01677,0.013711,2019:09:25 11:23:43,71,15.0,0.0,32.0
679,354972062885746,19.12041,73.01676,0.027819,2019:09:25 11:23:48,82,0.0,0.0,0.0
680,354972062885746,19.12015,73.01677,0.028930,2019:09:25 11:23:53,79,0.0,0.0,0.0


In [ ]:
73.01680,0.005658;73.01677,0.013711;73.01676,0.027819

In [27]:
Alerts = Alerts.reset_index(drop=True)

In [28]:
def remove_values_from_list(the_list, val):
      return [value for value in the_list if value != val]

In [29]:
Alerts['Alert']= [(each[5],each[6],each[7]) for each in Alerts.values] 
Alerts['Alert']=[remove_values_from_list(each, 0.0) for each in  Alerts.Alert]

In [30]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 10.903682470321655 seconds ---
